# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

In [3]:
for f in file_path_list:
    print(f)

/home/workspace/event_data/2018-11-27-events.csv
/home/workspace/event_data/2018-11-04-events.csv
/home/workspace/event_data/2018-11-07-events.csv
/home/workspace/event_data/2018-11-09-events.csv
/home/workspace/event_data/2018-11-19-events.csv
/home/workspace/event_data/2018-11-05-events.csv
/home/workspace/event_data/2018-11-22-events.csv
/home/workspace/event_data/2018-11-16-events.csv
/home/workspace/event_data/2018-11-26-events.csv
/home/workspace/event_data/2018-11-24-events.csv
/home/workspace/event_data/2018-11-29-events.csv
/home/workspace/event_data/2018-11-15-events.csv
/home/workspace/event_data/2018-11-20-events.csv
/home/workspace/event_data/2018-11-06-events.csv
/home/workspace/event_data/2018-11-18-events.csv
/home/workspace/event_data/2018-11-21-events.csv
/home/workspace/event_data/2018-11-10-events.csv
/home/workspace/event_data/2018-11-23-events.csv
/home/workspace/event_data/2018-11-02-events.csv
/home/workspace/event_data/2018-11-28-events.csv
/home/workspace/even

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 
session.execute("""
                CREATE KEYSPACE IF NOT EXISTS music_app
                WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor': 1} 
                """)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('music_app')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
# load data from csv to pandas dataframe
file = 'event_datafile_new.csv'
df = pd.read_csv(file, encoding='utf8')

### <mark>Query 1</mark>:  Give me the artist, song title and song's length in the music app history that was heard  (session_history)

In [10]:
query  =  "CREATE TABLE IF NOT EXISTS session_history "
query += "(session_id INT, session_item INT, artist TEXT, song TEXT, length FLOAT, \
           PRIMARY KEY(session_id, session_item))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
# load data to first table
for i, row in df.iterrows():
    query  = "INSERT INTO session_history (session_id, session_item, artist, song, length)"
    query += "VALUES(%s, %s, %s, %s, %s)"
    session.execute(query, (row.sessionId, row.itemInSession, row.artist, row.song, row.length))

In [12]:
query = "SELECT session_id, session_item, artist, song, length FROM session_history WHERE session_id=338 AND session_item=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns = ['session_id', 'session_item', 'artist', 'song', 'length'])


,session_id,session_item,artist,song,length
0,338,4,Faithless,Music Matters (Mark Knight Dub),495.307312


### <mark>Query 2</mark>: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 (user_history)

In [13]:
#table structure creation
query  =  "CREATE TABLE IF NOT EXISTS user_history"
query += "(user_id INT, session_id INT, session_item INT, artist TEXT, song TEXT, user TEXT, \
           PRIMARY KEY((user_id, session_id), session_item))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
#load data to the second table
for i, row in df.iterrows():
    query  = "INSERT INTO user_history (user_id, session_id, session_item, artist, song, user)"
    query += "VALUES(%s, %s, %s, %s, %s, %s)"
    session.execute(query, (row.userId, row.sessionId, row.itemInSession, row.artist, row.song, f'{row.firstName} {row.lastName}'))

In [15]:
query = "SELECT user_id, session_id, session_item, artist, song, user FROM user_history WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(query) #fogot rows initially
except Exception as e:
    print(e)

data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns = ['user_id', 'session_id', 'session_item', 'artist', 'song', 'user'])

,user_id,session_id,session_item,artist,song,user
0,10,182,0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,10,182,1,Three Drives,Greece 2000,Sylvie Cruz
2,10,182,2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,10,182,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


### <mark>Query 3</mark>: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' (song_history)


                    

In [16]:
query = "CREATE TABLE IF NOT EXISTS song_history"
query += "(song TEXT, user_id INT, PRIMARY KEY(song, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [17]:
# load data to third table

for i, row in df.iterrows():
    query = "INSERT INTO song_history (song, user_id)"
    query += "VALUES(%s, %s)"
    session.execute(query, (row.song, row.userId))

In [18]:
query = "SELECT song, user_id FROM song_history WHERE song='All Hands Against His Own'"

try:
    rows=session.execute(query)
except Exception as e:
    print(e)

data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns = ['song','user_id'])


,song,user_id
0,All Hands Against His Own,29
1,All Hands Against His Own,80
2,All Hands Against His Own,95


### Drop the tables before closing out the sessions

In [19]:
## TO-DO: Drop the table before closing out the sessions

In [20]:
session.execute("DROP TABLE IF EXISTS session_history")

In [21]:
session.execute("DROP TABLE IF EXISTS user_histry")

In [22]:
session.execute("DROP TABLE IF EXISTS song_history")

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()